In [2]:
import json
import psycopg2
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt

In [5]:
dbconfig = json.loads(open("res/dbconfig.properties").read())['cssc_jaunt']
dbconfig.pop("drivername")
dbconfig['user'] = dbconfig['username']
dbconfig.pop("username")
connection = psycopg2.connect(**dbconfig)
connection

<connection object at 0x000002EBA9042678; dsn: 'user=postgres password=xxx dbname=cssc host=cssc.hanyang.ac.kr port=5432', closed: 0>

In [6]:
article_df = pd.read_sql(sql = "SELECT \"title\" FROM article_entries WHERE \"aDate\" BETWEEN '2015-05-20' AND '2015-07-28 23:59:59'", con = connection)
article_df.to_csv("test.csv",index = False, encoding = 'utf-8')

In [7]:
def get_tags(text, ntags = 50):
    spliter = Okt()
    nouns = spliter.nouns(text)
    count = Counter(nouns)
    return_list = []
    
    for n, c in count.most_common(ntags):
        temp = {'tag': n, 'count': c}
        return_list.append(temp)

    return return_list

In [8]:
def main():
    noun_count = 20
    tags = get_tags(article_df.to_string(), noun_count)
    
    output_file_name = "count.txt"
    open_output_file = open(output_file_name, 'w', -1, "utf-8")

    for tag in tags:
        noun = tag['tag']
        count = tag['count']
        open_output_file.write('{} {}\n'.format(noun, count))
        
    open_output_file.close() 

if __name__ == '__main__':
    main()